In [ ]:
%pip install langchain_community
%pip install langchain
%pip install -qU langchain_ollama
%pip install sympy
%pip install google-genai
import pandas as pd
from sympy import sympify
from sympy.parsing.latex import parse_latex
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os
import sys
import json
import argparse
import pandas as pd
from google import genai

In [ ]:
Gemini_API_KEY = " Your-api-key"

In [ ]:
df = pd.read_csv("input.csv")    #Path to the instance-feature file for a specific algorithm
df["pred_solution_time"]  = None
df["pred_optimality_gap"] = None
df["pred_memory_kb"]      = None

In [103]:
def build_prompt(row):
    return (
        f"Supppose there's a Knapsack instance with number_of_elements={int(row['number_of_elements'])}, "
        f"capacity={row['capacity']}, "
        f"max_weight={row['max_weight']}, min_weight={row['min_weight']}, "
        f"mean_weight={row['mean_weight']}, median_weight={row['median_weight']}, std_weight={row['std_weight']}, "
        f"weight_range={row['weight_range']}, "
        F"max_profit={row['max_profit']}, min_profit={row['min_profit']}, "
        f"mean_profit={row['mean_profit']}, median_profit={row['median_profit']}, std_profit={row['std_profit']}, "
        f"profit_range={row['profit_range']}, "
        f"renting_ratio={row['renting_ratio']}, "
        f"mean_weight_profit_ratio={row['mean_weight_profit_ratio']}, "
        f"median_weight_profit_ratio={row['median_weight_profit_ratio']}, "
        f"capacity_mean_weight_ratio={row['capacity_mean_weight_ratio']}, "
        f"capacity_median_weight_ratio={row['capacity_median_weight_ratio']}, "
        f"capacity_std_weight_ratio={row['capacity_std_weight_ratio']}, "
        f"std_weight_profit_ratio={row['std_weight_profit_ratio']}, "
        f"weight_profit_correlation={row['weight_profit_correlation']}, "
        f"cpu_cores={int(row['cpu_cores'])}, ram={int(row['ram'])}GB. "
        "Based on these features, predict the **solution_time** (in milliseconds), **optimality_gap** (fraction), "
        "**peak_memory** (in KB) if solved using the **branch and bound** algorithm for profit-maximization in 0-1 knapsack. "
        "Consider time-limit set for the algorithm to run is 300seconds. "
        "Return **only** a JSON object like :\n "
        '{"solution_time": <double>, "optimality_gap": <double>, "memory_kb": <int>}.'
    )


In [104]:
def strip_code(s: str) -> str:
  
    lines = s.strip().splitlines()

    if lines and lines[0].lstrip().startswith("```"):
        lines = lines[1:]
    if lines and lines[-1].lstrip().startswith("```"):
        lines = lines[:-1]
    return "\n".join(lines)

In [ ]:
import re
import json

def extract_json(text):
    m = re.search(r'```json\s*({.*?})\s*```', text, flags=re.DOTALL)
    if m:
        return m.group(1)
    start = text.find('{')
    end   = text.rfind('}')
    if start != -1 and end != -1:
        return text[start:end+1]
    return None

In [ ]:
import time
from tqdm import tqdm
from google import genai
import google.genai.errors as genai_errors
import json
import pandas as pd

client = genai.Client(api_key="Your-api-key")

output_path = "output.csv"


df_head = pd.DataFrame(columns=df.columns)
df_head.to_csv(output_path, index=False)

for idx, row in tqdm(df.iterrows(), total=len(df)):
    prompt = build_prompt(row)
    try:
        resp  = client.models.generate_content(
            model="gemini-2.5-flash-preview-05-20",
            contents=prompt
        )
        raw   = resp.text
        clean = strip_code(raw)

    
        try:
            out = json.loads(clean)
        except json.JSONDecodeError:
            # Fallback to extracting a JSON substring
            jstr = extract_json(raw)
            if jstr:
                out = json.loads(jstr)
            else:
                out = {}

        # Assign predictions (will be None if missing)
        df.at[idx, "pred_solution_time"]  = out.get("solution_time")
        df.at[idx, "pred_optimality_gap"] = out.get("optimality_gap")
        df.at[idx, "pred_memory_kb"]      = out.get("memory_kb")

    except genai_errors.ServerError as e:
        print(f"[Row {idx}] ServerError: {e}. Assigning None and skipping.")
        df.at[idx, "pred_solution_time"]  = None
        df.at[idx, "pred_optimality_gap"] = None
        df.at[idx, "pred_memory_kb"]      = None

    
    df.iloc[[idx]].to_csv(output_path, mode='a', header=False, index=False)

 16%|█▌        | 8/50 [02:33<12:16, 17.53s/it]

[Row 7] ServerError: 502 Bad Gateway. {'message': '<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 502 (Server Error)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp

100%|██████████| 50/50 [18:11<00:00, 21.82s/it]
